# Почему L1 (Lasso) регуляризация отбирает признаки?

# L1 и L2 регуляризация

Вспомним две базовые техники регуляризации.

Для простоты будет разбирать все на основе линейной регрессии со
среднеквадратичной ошибкой.

Функция потерь с L1 регуляризацией:

$$ \text{Loss}+ L_1 = \frac{1}{2n} ||y - Xw||_{2}^{2} + \alpha ||w||_1 $$

$$ ||w||_1 = |w_1| + |w_2| + \ldots $$

И с L2 регуляризацией:

$$ \text{Loss} + L_2 = \frac{1}{2n} ||y - Xw||_{2}^{2} + \alpha ||w||_{2}^{2} $$

$$ ||w||_{2}^{2} = w_{1}^2 + w_{2}^2 + \ldots $$

L1 обычно рекомендуют использовать в ситуации, когда:

-   в датасете огромное количество признаков
-   заранее известно, что часть признаков неинформативна
-   вычислительные ресурсы ограничены (нужно очень быстро получать ответ
    или данных очень много)

В процессе оптимизации L1 техника регуляризации полностью обнуляет часть
признаков и в итоговой модели их можно не использовать.

Ага, понятно… Стоп, а как обнуляет-то?

Не особо очевидно. Будет разбираться.

-   [ ] (или можно сразу прыгнуть к итогам → ссылка)

# Спускаемся по градиенту

Вообще, L1 просто константу добавляет/отнимает при градиентном спуске:

$$ \frac{\partial L_1}{\partial w_i} = \alpha \cdot \text{sign}(w_i) $$

$$w_i := w_i - \lambda (\frac{\partial Loss}{\partial w_i} +  \alpha \cdot \text{sign}(w_i) ) $$

Интуитивно, обнуляемые веса будут болтаться около ноля, но вот ровно в
ноль попадать практически никогда не будут. Ну что ж, проверим интуицию
на практике.

## Генеририруем простенький датасет

-   50 объектов
-   10 признаков со случайным шумом
-   3 признака с нулем
-   1 признак с целевой переменной + немного шума
-   последний признак константа с 1 (bias)

In [1]:
# %% Generate data
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso


np.random.seed(42)

y = np.random.randn(50, 1)

X = np.random.randn(50, 10)
X = np.hstack((X, np.zeros((X.shape[0], 3))))
X = np.hstack((X, y + 0.01 * np.random.randn(50, 1)))

bias_column = np.ones((X.shape[0], 1))
X = np.hstack((X, bias_column))
print("Пример объекта: ")
print(X[0])

## Реализуем градиентный спуск для L1

In [2]:
from sklearn.metrics import mean_squared_error

n_samples, n_features = X.shape

np.random.seed(42)

class LinearRegressionNaiveL1:
    def __init__(self, learning_rate=1e-5, iterations=150, alpha=5):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.alpha = alpha  # regularization parameter
        self.weights = np.random.randn(n_features, 1) * 0.001
        self.bias = 0
        self.loss_history = []

    def fit(self, X, y):
        n_samples, n_features = X.shape

        for _ in range(self.iterations):
            # Compute predictions
            y_pred = self.predict(X)

            # Compute gradients
            d_l1 = self.alpha * np.sign(self.weights)
            d_weights = (1/(2*n_samples)) * (X.T @ (y_pred - y))
            d_bias = (1/(2*n_samples)) * np.sum(y_pred - y)

            # Update weights and bias
            self.weights -= self.learning_rate * (d_weights + d_l1)
            self.bias -= self.learning_rate * d_bias
            
            mse = mean_squared_error(y, y_pred)            
            self.loss_history.append(mse)

    def predict(self, X):
        return X @ self.weights + self.bias

In [3]:
model = LinearRegressionNaiveL1()
model.fit(X, y)

plt.plot(model.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Mean Squared Error')
plt.title('Loss Change Over Iterations')
plt.show()


plt.bar(range(len(model.weights)), model.weights.flatten())
plt.xlabel('Feature Index')
plt.ylabel('Weight Value')
plt.title('Weights')
plt.show()

# Проверим, что в общеупотребительное решение, что работает

От теории к практике. Как l1 и l2 работают в sklearn

L1 регуляризация действительно полностью обнуляет часть весов модели, в
отличии от L2 регуляризации.

Мы можем это проверить,

## сгенерировать простой датасет

мы знаем как он себя ведет

дальше будем работать только с ним

``` python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# import warnings
# warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')


# Generate synthetic data
np.random.seed(42)
X = np.random.randn(100, 10)
y = X @ np.array([1.5, -2, 0, 0, 4.5, 0, 0, -3.5, 2, 0]) + np.random.randn(100)

# Fit a linear regression model with L2 regularization
lr = Ridge()
lr.fit(X, y)
weights_lr = lr.coef_

# Fit a linear regression model with L1 regularization
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)
weights_lasso = lasso.coef_

# Plot the weights for comparison
# plt.figure(figsize=(10, 5))
plt.plot(weights_lr, 'o-', label='Ridge Regression')
plt.plot(weights_lasso, 's-', label='Lasso Regression')
plt.xlabel('Feature index')
plt.ylabel('Weight value')
plt.title('Comparison of Weights (sklearn lib): Ridge Regression vs Lasso Regression')
plt.legend()
plt.grid(True)
plt.show()

# Display the weights
print("Ridge Regression Weights:\n", weights_lr)
print("Lasso Regression Weights:\n", weights_lasso)
```

Рисунок 1

# Л1 наивная реализация

код

не работает… почему?

есть три нюанса: 1. коэффициэнты стратуеют с нуля (для линейной модели
можно) 2. используется модификация градиентного спуска 3. ?

# Л1 тру реализация

дело не только в производной л1

используется модификация градиентного спуска

сам метод можно вывести через субградинеты, но это усложнение

# Итоги

Можно понять, что Л1 далеко не всегда является хорошей штукой

# Прочие объяснения

Обычно Л1 объясняют через вот эти графики, но мне оно не очевидно

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Generate the original meshgrid
x = np.linspace(-2, 2, 800)
y = np.linspace(-2, 2, 800)
X, Y = np.meshgrid(x, y)

# L1 norm
Z1 = np.abs(X) + np.abs(Y)

# L2 norm
Z2 = np.sqrt(X**2 + Y**2)

# Rotation angle in radians
theta = np.pi * 0.25 

# Rotate the coordinates
X_rot = X * np.cos(theta) - Y * np.sin(theta)
Y_rot = X * np.sin(theta) + Y * np.cos(theta)

# MSE with target (a, b)
a, b = 1, 0
Z_mse_rot = (X_rot - a)**2 + (2*Y_rot - b)**2

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# L1 contour plot
ax1.contour(X, Y, Z1, levels=10)
ax1.set_title('L1 Norm Contour Plot')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.axhline(0, color='black', linewidth=0.5)
ax1.axvline(0, color='black', linewidth=0.5)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)

# L2 contour plot
ax2.contour(X, Y, Z2, levels=10)
ax2.set_title('L2 Norm Contour Plot')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.axhline(0, color='black', linewidth=0.5)
ax2.axvline(0, color='black', linewidth=0.5)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)

# MSE contour plot
ax3.contour(X, Y, Z_mse_rot, levels=10)
ax3.set_title('MSE Contour Plot')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.axhline(0, color='black', linewidth=0.5)
ax3.axvline(0, color='black', linewidth=0.5)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

plt.show()

In [6]:
fig, ax = plt.subplots(figsize=(8, 8))

# L1 norm contour plot
ax.contour(X, Y, Z1, levels=[1], colors='b', label='L1 Norm')

# L2 norm contour plot
ax.contour(X, Y, Z2, levels=[1], colors='g', label='L2 Norm')

# MSE contour plot
ax.contour(X, Y, Z_mse_rot, levels=[1], colors='r', label='MSE')

ax.set_title('Contour Plots of L1, L2 Norms, and MSE')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=0.5)
ax.grid(color='gray', linestyle='--', linewidth=0.5)
ax.legend()

plt.tight_layout()
plt.show()

In [7]:
import numpy as np
import matplotlib.pyplot as plt

# Generate the original meshgrid
x = np.linspace(-2, 2, 800)
y = np.linspace(-2, 2, 800)
X, Y = np.meshgrid(x, y)

# L1 norm
Z1 = np.abs(X) + np.abs(Y)

# L2 norm
Z2 = np.sqrt(X**2 + Y**2)

# Rotation angle in radians
theta = np.pi * 0.25 

# Rotate the coordinates
X_rot = X * np.cos(theta) - Y * np.sin(theta)
Y_rot = X * np.sin(theta) + Y * np.cos(theta)

# MSE with target (a, b)
a, b = 1, 0
Z_mse_rot = (X_rot - a)**2 + (4*Y_rot - b)**2

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# L1 contour plot
ax1.contour(X, Y, Z1, levels=10)
ax1.set_title('L1 Norm Contour Plot')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.axhline(0, color='black', linewidth=0.5)
ax1.axvline(0, color='black', linewidth=0.5)
ax1.grid(color='gray', linestyle='--', linewidth=0.5)

# L2 contour plot
ax2.contour(X, Y, Z2, levels=10)
ax2.set_title('L2 Norm Contour Plot')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.axhline(0, color='black', linewidth=0.5)
ax2.axvline(0, color='black', linewidth=0.5)
ax2.grid(color='gray', linestyle='--', linewidth=0.5)

# MSE contour plot
ax3.contour(X, Y, Z_mse_rot, levels=10)
ax3.set_title('MSE Contour Plot')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.axhline(0, color='black', linewidth=0.5)
ax3.axvline(0, color='black', linewidth=0.5)
ax3.grid(color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

plt.show()

In [8]:
fig, ax = plt.subplots(figsize=(8, 8))

# L1 norm contour plot
ax.contour(X, Y, Z1, levels=[1], colors='b', label='L1 Norm')

# L2 norm contour plot
ax.contour(X, Y, Z2, levels=[1], colors='g', label='L2 Norm')

# MSE contour plot
ax.contour(X, Y, Z_mse_rot, levels=[1], colors='r', label='MSE')

ax.set_title('Contour Plots of L1, L2 Norms, and MSE')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(0, color='black', linewidth=0.5)
ax.grid(color='gray', linestyle='--', linewidth=0.5)
ax.legend()

plt.tight_layout()
plt.show()